## Умова


### Хід виконання роботи

1. Завантажити набiр кольорових зображень з попереднього практикуму згiдно з
варiантом. Якщо набiр великого розмiру - можна обрати частину.
2. Пiдготувати данi для навчання. Виконати аугментацiю даних.
3. Роздiлити данi на навчальну, валiдацiйну i тестову пiдмножини.
4. Побудувати моделi класифiкацiї зображень на основi попередньо навчених глибоких згорткових мереж, використовуючи технологiю передачi знань (transfer learning):
    * Завантажити попередньо навченi ваги. Iмпортувати ваги, отриманi пiд час навчання обраних глибоких моделей на наборi зображень ImageNet.
    * Побудувати один або кiлька верхнiх повнозв’язних шарiв. Останнiм (результуючим) шаром мережi має бути повнозв’язний softmax-шар з кiлькiстю нейронiв, яка дорiвнює кiлькостi класiв в заданому за варiантом наборi даних.
    * Заморозити попередньо навченi ваги. Заморожуючи змiннi попередньої моделi ми гарантуємо, що буде навчатися тiльки один (кiлька) верхнiх повнозв’язних шарiв; значення попередньої моделi залишаться незмiнними.
    * Виконати донавчання доданих верхнiх шарiв на власному наборi зображень.
    * Налаштувати параметри доданих верхнiх шарiв на валiдацiйнiй пiдмножинi. Для дослiдження обрати кiлька попередньо навчених глибоких моделей, наприклад з наступних: VGG19, Xception, InceptionV3, ResNet152, DenseNet201, EfficientNetB7.
5. Вiдобразити у TensorBoard графiки, якi iлюструють оцiнки якостi навчання моделей:
    * графiки змiни функцiї втрат на тренувальнiй i валiдацiйнiй множинах по мiрi навчання моделей,
    * графiки змiни показникiв якостi моделi (accuracy, f1-score, AUC) на тренувальнiй i валiдацiйнiй множинах по мiрi навчання моделей.
6. Розрахувати на тестовiй множинi оцiнки якостi обраної найкращої моделi.
7. Завантажити зображення тестової множини i розпiзнати його навченими моделями.
8. Зробити висновки щодо якостi класифiкацiї на основi побудованих моделей. Порiвняти з результатами попереднього практикуму.


### Варіант

11. The Street View House Number

## Програмна реалізація

### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import tensorflow as tf

from sklearn.model_selection import train_test_split


In [ ]:
# Load the TensorBoard notebook extension
# %load_ext tensorboard

In [2]:
# Clear any logs from previous runs
!rm -rf ./logs/

### EDA The Street View House Number


Було детально розглянуто в lab4.


In [3]:
batch_size = 256

num_classes = 10

# img_height = 32
# img_width = 32
num_channels = 3

In [4]:
!wget -q http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget -q http://ufldl.stanford.edu/housenumbers/test_32x32.mat

In [5]:
train = scipy.io.loadmat('train_32x32.mat')
test = scipy.io.loadmat('test_32x32.mat')

In [6]:
X_train = np.transpose(train["X"]/255.,(3,0,1,2))
y_train = train["y"]
X_test = np.transpose(test["X"]/255.,(3,0,1,2))
y_test = test["y"]

In [7]:
for i, label in enumerate(y_train):
    if label == 10:
        y_train[i][0] = 0

for i, label in enumerate(y_test):
    if label == 10:
        y_test[i][0] = 0

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [8]:
X_test, X_val, y_test, y_val = train_test_split(
    X_test,
    y_test,
    stratify=y_test,
    random_state=42
)

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(batch_size * 10).batch(batch_size).prefetch(buffer_size=AUTOTUNE)
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(buffer_size=AUTOTUNE)
val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [10]:
img_height, img_width = 224, 224

def resize_images(x, y):
    x = tf.image.resize(x, [img_height, img_width])
    return x, y

train_data = train_data.map(resize_images)
val_data = val_data.map(resize_images)
test_data = test_data.map(resize_images)


In [11]:
augmentation_layers = [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
]


def data_augmentation(x):
    for layer in augmentation_layers:
        x = layer(x)
    return x

train_data = train_data.map(lambda x, y: (data_augmentation(x), y))

### Train model

In [14]:
def create_model(base_model):
    base_model.trainable = False

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    return tf.keras.Model(base_model.inputs, outputs)


model = {}
model["MobileNet"] = create_model(
    tf.keras.applications.MobileNet(weights="imagenet", include_top=False,input_shape=(img_height, img_width, num_channels)))
model["EfficientNetB0"] = create_model(
    tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False,input_shape=(img_height, img_width, num_channels)))


In [ ]:
history = {}

for model_name in model.keys():
    model[model_name].compile(
        optimizer=tf.keras.optimizers.Adam(0.01),
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy']
    )

    print(f"fit {model_name}...")
    history[model_name] = model[model_name].fit(
        train_data,
        validation_data=val_data,
        epochs=2,
        verbose=1,
    )
    print()
    print(f"evaluate {model_name}...")
    model[model_name].evaluate(test_data,verbose=1)
    print()

fit MobileNet...
Epoch 1/2
287/287 ━━━━━━━━━━━━━━━━━━━━ 570s 2s/step - categorical_accuracy: 0.5139 - loss: 1.4489 - val_categorical_accuracy: 0.7041 - val_loss: 0.9322
Epoch 2/2
287/287 ━━━━━━━━━━━━━━━━━━━━ 544s 2s/step - categorical_accuracy: 0.6585 - loss: 1.0370 - val_categorical_accuracy: 0.6832 - val_loss: 0.9903

evaluate MobileNet...
77/77 ━━━━━━━━━━━━━━━━━━━━ 27s 343ms/step - categorical_accuracy: 0.6718 - loss: 1.0135

fit EfficientNetB0...
Epoch 1/2
287/287 ━━━━━━━━━━━━━━━━━━━━ 636s 2s/step - categorical_accuracy: 0.1760 - loss: 2.2558 - val_categorical_accuracy: 0.1959 - val_loss: 2.2423
Epoch 2/2
287/287 ━━━━━━━━━━━━━━━━━━━━ 548s 2s/step - categorical_accuracy: 0.1809 - loss: 2.2563 - val_categorical_accuracy: 0.1959 - val_loss: 2.2416

evaluate EfficientNetB0...
77/77 ━━━━━━━━━━━━━━━━━━━━ 31s 402ms/step - categorical_accuracy: 0.1970 - loss: 2.2418



### Пошук найкращої моделі

В даному випадку, найкращі результати показали більш складні архітектури, що тренувалися як на кольорових, так і ЧБ картинках. Непогано зарекомендували себе і базові моделі з одним згортковим шаром.

In [ ]:
random_samples = test_data.unbatch().shuffle(buffer_size=1000).take(20)
samples = list(random_samples)
features = np.array([img.numpy() for img, _ in samples])
labels = np.array([label.numpy() for _, label in samples])

In [ ]:
predict_MobileNet = model["MobileNet"].predict(features, verbose=0)
predict_EfficientNetB0 = model["EfficientNetB0"].predict(features, verbose=0)

fig, axes = plt.subplots(4, 5, figsize=(12, 16))

for i, (img, label) in enumerate(samples):
    ax = axes[i // 5, i % 5]
    ax.imshow(img.numpy())
    title = f"True Label:{np.argmax(label)}"
    title += "\n" + f"MobileNet: {np.argmax(predict_MobileNet[i])} (p={np.max(predict_MobileNet[i]):.4f})"
    title += "\n" + f"EfficientNetB0: {np.argmax(predict_EfficientNetB0[i])} (p={np.max(predict_EfficientNetB0[i]):.4f})"
    ax.set_title(title)
    ax.axis('off')

plt.tight_layout()
plt.show()

## Висновки

Хоч в даній роботі не було досліджено багато різних параметрів згорткових шарів та вплив шарів дропауту, чи батчнормалізації, зрозуміло, що вони як дають перевагу у точносні, та зменшенні перенавчання, проте можуть дещо затрудняти навчання моделі, або ж навпаки. Окрім цього, колірна схема майже не впливає на точність та час навчання моделі.